This notebook is meant to demonstrate the transformation of an annotated notebook into a HTTP API using the Jupyter kernel gateway. The result is a simple scotch recommendation engine.

The original scotch data is from [https://www.mathstat.strath.ac.uk/outreach/nessie/nessie_whisky.html](https://www.mathstat.strath.ac.uk/outreach/nessie/nessie_whisky.html).

In [1]:
Pkg.add("JSON")
import JSON
Pkg.add("Requests")
using Requests
import Requests: get

INFO: Nothing to be done
INFO: METADATA is out-of-date — you may not have the latest version of JSON
INFO: Use `Pkg.update()` to get the latest versions of your packages
INFO: Nothing to be done
INFO: METADATA is out-of-date — you may not have the latest version of Requests
INFO: Use `Pkg.update()` to get the latest versions of your packages
and contains known security vulnerabilities. Please upgrade to a
more recent version.


## Data

We read the scotch data from a public Dropbox URL to make this notebook more portable. This is acceptable for small, public, demo data which is what we have here.

In [2]:
scotch_json_response = get("https://dl.dropboxusercontent.com/u/19043899/whisky_features_df.json")
scotch_json = JSON.parse(IOBuffer(scotch_json_response.data))
scotch_similarity_json_reponse = get("https://dl.dropboxusercontent.com/u/19043899/whisky_similarity_features_df.json")
scotch_similarity_json = JSON.parse(IOBuffer(scotch_similarity_json_reponse.data))

86-element Array{Any,1}:
 ["Tormore"=>0.8121035618,"Teaninich"=>0.7499402176,"ArranIsleOf"=>0.8433383142,"Distillery"=>"Aberfeldy","Miltonduff"=>0.8155908717,"Auchentoshan"=>0.7884045093,"Bladnoch"=>0.826961254,"Craigallechie"=>0.9184854354,"Glenkinchie"=>0.828849727,"Speyside"=>0.9444501378,"Craigganmore"=>0.8589556904,"BlairAthol"=>0.9756097561,"Linkwood"=>0.8711400676,"Ardmore"=>0.9252891981,"Inchgower"=>0.8414890702,"Laphroig"=>0.5426095162,"Glenlivet"=>0.9467531164,"Benrinnes"=>0.9653452203,"Glenrothes"=>0.8971499589,"Tobermory"=>0.8834522086,"Bruichladdich"=>0.8946750578,"GlenGarioch"=>0.8148353451,"AnCnoc"=>0.888956637,"Belvenie"=>0.8946750578,"Dufftown"=>0.8120207068,"BenNevis"=>0.9096109718,"Cardhu"=>0.8839122202,"Dailuaine"=>0.927624819,"Ardbeg"=>0.5692985756,"Balmenach"=>0.899820054,"GlenElgin"=>0.911013611,"RoyalLochnagar"=>0.962222837,"Tullibardine"=>0.7983723279,"Auchroisk"=>0.9630363567,"Highland Park"=>0.9136498769,"Benriach"=>0.8447418838,"Glendronach"=>0.899820054,"Gl

In [3]:
scotch_matrix = Any[]
for i in scotch_json
    temp = Any[]
    for j in values(i)
        push!(temp, j)
    end
    push!(scotch_matrix, temp)
end
scotch_matrix = reshape(scotch_matrix, 86,1)

86x1 Array{Any,2}:
 {"Aberfeldy",2,2,0,2,1,0,2,2,2,2,2,2}   
 {"Aberlour",3,1,0,3,3,0,3,4,2,3,2,2}    
 {"AnCnoc",1,2,0,3,0,0,2,2,1,3,0,2}      
 {"Ardbeg",4,4,0,1,2,4,2,0,0,1,0,1}      
 {"Ardmore",2,2,0,2,1,0,3,1,3,1,1,2}     
 {"ArranIsleOf",2,1,0,3,1,1,1,1,1,1,1,0} 
 {"Auchentoshan",0,0,0,2,1,0,2,1,1,3,0,2}
 {"Auchroisk",2,1,0,3,1,0,2,2,2,2,2,2}   
 {"Aultmore",2,1,0,2,0,0,2,1,1,2,0,2}    
 {"Balblair",2,2,0,3,2,1,1,0,3,2,0,2}    
 {"Balmenach",4,2,0,3,1,0,0,2,2,1,3,3}   
 {"Belvenie",3,1,0,2,2,0,2,3,2,2,1,0}    
 {"BenNevis",4,2,0,2,2,0,2,2,2,2,0,2}    
 ⋮                                       
 {"Springbank",2,2,0,2,2,2,1,2,3,0,1,2}  
 {"Strathisla",2,1,0,2,2,0,3,2,2,3,2,3}  
 {"Strathmill",2,1,0,3,2,0,1,0,1,3,0,2}  
 {"Talisker",4,3,0,2,3,3,2,1,0,2,0,1}    
 {"Tamdhu",1,1,0,2,0,0,2,2,1,2,1,1}      
 {"Tamnavulin",1,2,0,3,2,0,1,0,1,2,0,2}  
 {"Teaninich",2,2,0,2,2,1,0,0,3,2,0,0}   
 {"Tobermory",1,1,0,1,0,0,2,1,1,2,0,1}   
 {"Tomatin",2,2,0,3,2,0,2,2,3,0,1,1}     
 {"Tomintoul",0

## API

We need to define a global REQUEST JSON string that will be replaced on each invocation of the API. We only care about path parameters and query string arguments, so we default those to blank here for development.

In [4]:
REQUEST = ""

""

Provide a way to get the names of all the scotches known by the model.

In [5]:
# GET /scotches
scotches = String[]
for i in scotch_matrix
    push!(scotches, i[1])
end
println(JSON.json(scotches))

["Aberfeldy","Aberlour","AnCnoc","Ardbeg","Ardmore","ArranIsleOf","Auchentoshan","Auchroisk","Aultmore","Balblair","Balmenach","Belvenie","BenNevis","Benriach","Benrinnes","Benromach","Bladnoch","BlairAthol","Bowmore","Bruichladdich","Bunnahabhain","Caol Ila","Cardhu","Clynelish","Craigallechie","Craigganmore","Dailuaine","Dalmore","Dalwhinnie","Deanston","Dufftown","Edradour","GlenDeveronMacduff","GlenElgin","GlenGarioch","GlenGrant","GlenKeith","GlenMoray","GlenOrd","GlenScotia","GlenSpey","Glenallachie","Glendronach","Glendullan","Glenfarclas","Glenfiddich","Glengoyne","Glenkinchie","Glenlivet","Glenlossie","Glenmorangie","Glenrothes","Glenturret","Highland Park","Inchgower","Isle of Jura","Knochando","Lagavulin","Laphroig","Linkwood","Loch Lomond","Longmorn","Macallan","Mannochmore","Miltonduff","Mortlach","Oban","OldFettercairn","OldPulteney","RoyalBrackla","RoyalLochnagar","Scapa","Speyburn","Speyside","Springbank","Strathisla","Strathmill","Talisker","Tamdhu","Tamnavulin","Teani

##### get_scotch_feautre_tuple:
Input = scotch_requested: String<br>
Output = Tuple of (scotch_requested feature names array, scotch_requested feature values array)

In [6]:
function get_scotch_feature_tuple(scotch_requested)
    #filter scotch matrix to the row where scotch_requested equals the first value/Distillery
    scotch_request_values = filter(row -> row[1] == scotch_requested, scotch_matrix)[1]
    scotch_request_keys = Any[]
    #create array of scotch feature names as the keys
    for i in scotch_json[1]
        push!(scotch_request_keys, i[1])
    end
    return (scotch_request_keys, scotch_request_values)
end

get_scotch_feature_tuple (generic function with 1 method)

Let clients query for features about a specific scotch given its name.

In [7]:
# GET /scotches/:scotch
#REQUEST = "{\"args\": {}, \"body\": \"\", \"path\": {\"scotch\": \"AnCnoc\"}}"
request_json = JSON.parse(REQUEST)
scotch_requested = "Talisker"
if haskey(request_json, "path") && haskey(request_json["path"], "scotch")
    scotch_requested = request_json["path"]["scotch"]
end
scotch_requested_tuple = get_scotch_feature_tuple(scotch_requested)
scotch_requested_features = Dict(zip(scotch_requested_tuple[1], scotch_requested_tuple[2]))
println(JSON.json(scotch_requested_features))

{"Distillery":"AnCnoc","Body":1,"Smoky":2,"Tobacco":0,"Sweetness":3,"Spicy":0,"Medicinal":0,"Malty":2,"Honey":2,"cluster":1,"Fruity":3,"Winey":0,"Nutty":2}


Let clients request a set of scotches similar to the one named. Let clients specify how many results they wish to receive (count) and if they want all of the raw feature data included in the result or not (include_features).

In [8]:
# GET /scotches/:scotch/similar
#REQUEST = "{\"args\": {\"count\": 4, \"include_features\": \"True\"}, \"body\": \"\", \"path\": {\"scotch\": \"AnCnoc\"}}"
request_json = JSON.parse(REQUEST)
scotch_requested = "Talisker"
num_results = 5
features_included = false

if haskey(request_json, "path") && haskey(request_json["path"], "scotch")
    scotch_requested = request_json["path"]["scotch"]
end
if haskey(request_json, "args")
    if haskey(request_json["args"], "count")
        num_results = request_json["args"]["count"]
    end
    if haskey(request_json["args"], "include_features")
        features_included = "True" == request_json["args"]["include_features"]
    end
end     

scotch_request_row = filter(row -> row["Distillery"] == scotch_requested, scotch_similarity_json)[1]
scotch_request_types = String[]
scotch_request_similarities = Float64[]
for (key, value) in scotch_request_row
    if key == "Distillery"
        continue
    end
    push!(scotch_request_types, key)
    push!(scotch_request_similarities, value)
end

#sort array of tuples by scotch similarity values
top_similiar_scotches = sort(collect(zip(scotch_request_similarities, scotch_request_types)), rev=true)
#subset to the top requested results
top_similiar_scotches = top_similiar_scotches[2:2+num_results-1]
#swap the ordering of the tuples in the array of tuples
top_similiar_scotches = map( x -> (x[2],x[1]), top_similiar_scotches)

if features_included
    #for each of the top similiar scotches
    for (distillery, similarity)  in top_similiar_scotches
        #get features of this particular scotch a tuple of arrays
        #where tuple[1] is feature name array, tuple[2] is feature value array
        scotch_feature_tuple = get_scotch_feature_tuple(distillery)
        push!(scotch_feature_tuple[1], "Similarity")
        push!(scotch_feature_tuple[2], float(similarity))
        #zip the feature arrays and convert to a Dict, so as to be converted to JSON easily
        similiar_scotches_with_features = Dict(zip(scotch_feature_tuple[1], scotch_feature_tuple[2]))
        println(JSON.json(similiar_scotches_with_features))
    end
else
    #convert array of tuples to an array of dicts where each value has a map of feature name and feature value
    println(JSON.json(map(x -> ["Distillery" => x[1], "Similarity" => x[2]], top_similiar_scotches)))
end

{"Distillery":"Cardhu","Body":1,"Smoky":1,"Tobacco":0,"Sweetness":3,"Spicy":1,"Medicinal":0,"Malty":2,"Honey":1,"cluster":1,"Fruity":2,"Similarity":0.9526279442,"Winey":0,"Nutty":2}
{"Distillery":"Tamdhu","Body":1,"Smoky":1,"Tobacco":0,"Sweetness":2,"Spicy":0,"Medicinal":0,"Malty":2,"Honey":2,"cluster":1,"Fruity":2,"Similarity":0.9486832981,"Winey":1,"Nutty":1}
{"Distillery":"Glenallachie","Body":1,"Smoky":1,"Tobacco":0,"Sweetness":3,"Spicy":1,"Medicinal":0,"Malty":2,"Honey":1,"cluster":1,"Fruity":2,"Similarity":0.9432999601,"Winey":0,"Nutty":1}
{"Distillery":"Aultmore","Body":2,"Smoky":1,"Tobacco":0,"Sweetness":2,"Spicy":0,"Medicinal":0,"Malty":2,"Honey":1,"cluster":1,"Fruity":2,"Similarity":0.9432999601,"Winey":0,"Nutty":2}
